In [79]:
import numpy as np
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer
import sys
import os
stopwords = feature_extraction.text.ENGLISH_STOP_WORDS
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatiser = WordNetLemmatizer()
from string import punctuation
from nltk.tokenize import word_tokenize
import operator
import math
from sklearn.decomposition import LatentDirichletAllocation
import re
import random
import bs4
from bs4 import BeautifulSoup
from soft_clustering_measure import *

In [68]:
text_file = open("/Users/panktibhalani/Downloads/whole_dataset.txt", "r");
lines = text_file.readlines();
labels = []
doctext = []
for line in lines:
    line = line[1:-1]
    listoflines = line.split(',', 2)
    doctext.append(listoflines[2].strip())
    labels.append(listoflines[1].strip())

In [73]:
len(labels)

18846

In [3]:
from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [4]:
def removeStopwords(wordlist):
    return [w for w in wordlist if w not in stopwords]
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [5]:
def getwords(sentence):
    sentence = strip_punctuation(sentence.lower())
    list_of_words = sentence.split();
    list_of_words = removeStopwords(list_of_words);
    return list_of_words

In [6]:
def lematized_word(postaged,word):
    try:
        postager = get_wordnet_pos(postaged[word])
        if(postager == ''):
            lmzword = lemmatiser.lemmatize(word)
        else:
            lmzword = lemmatiser.lemmatize(word,pos=postager)
    except:
        lmzword = lemmatiser.lemmatize(word)
    return lmzword

In [71]:
document = []
for doc in doctext:
    doc = doc.strip()
    doc.replace("\n"," ")
    sentences = re.split("[!.]",doc)
    wordlist = []
    for sentence in sentences:
        postaged = dict(pos_tag(word_tokenize(sentence)))
        list_of_words = getwords(sentence)
        for word in list_of_words:
            lmzword = lematized_word(postaged,word)
            wordlist.append(lmzword)
    document.append(wordlist)

In [72]:
len(document)

18846

In [74]:
cleanedDoc = []
for doc in document:
    cleanedDoc.append(" ".join(doc))

In [13]:
def TopDocs(model, feature_names, no_top_words):
    Topics=[]
    for topicid, topic in enumerate(model.components_):
        topic_norm=np.linalg.norm(topic)
        if topic_norm != 0:
            topic=topic/topic_norm
        Topics.append(topic)
    return np.array(Topics)

In [75]:
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
termfq = count_vectorizer.fit_transform(cleanedDoc)
count_feature_names = count_vectorizer.get_feature_names()

In [76]:
no_topics = 50
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(termfq)

In [80]:
result = lda.transform(termfq)

In [81]:
result.shape

(18846, 50)

In [82]:
setoflabels = set(labels)

In [83]:
len(setoflabels)

20

In [105]:
dictla = {}
ind = 0
for l in setoflabels:
    dictla[l] = ind
    ind += 1

In [106]:
dictla

{"'alt.atheism'": 8,
 "'comp.graphics'": 2,
 "'comp.os.ms-windows.misc'": 6,
 "'comp.sys.ibm.pc.hardware'": 9,
 "'comp.sys.mac.hardware'": 17,
 "'comp.windows.x'": 14,
 "'misc.forsale'": 18,
 "'rec.autos'": 16,
 "'rec.motorcycles'": 15,
 "'rec.sport.baseball'": 3,
 "'rec.sport.hockey'": 0,
 "'sci.crypt'": 13,
 "'sci.electronics'": 4,
 "'sci.med'": 5,
 "'sci.space'": 11,
 "'soc.religion.christian'": 19,
 "'talk.politics.guns'": 7,
 "'talk.politics.mideast'": 12,
 "'talk.politics.misc'": 1,
 "'talk.religion.misc'": 10}

In [84]:
text_file = open("/Users/panktibhalani/Downloads/sample_dataset.txt", "r");
lines = text_file.readlines();
indexestemp = []
for line in lines:
    lines= line.split(",")
    indexestemp.append(int(lines[0][1:]) - 1)

In [85]:
filtereddata = result[indexestemp]

In [90]:
len(filtereddata)

4491

In [87]:
from sklearn.mixture import GaussianMixture

gmmresultfsh = GaussianMixture(n_components=20, covariance_type='diag').fit(filtereddata)
print(gmmresultfsh.converged_)
print(gmmresultfsh.covariances_.shape)
print(gmmresultfsh.means_.shape)
print(gmmresultfsh.weights_)

True
(20, 50)
(20, 50)
[0.0747084  0.02841395 0.03520461 0.05381435 0.06264997 0.03246752
 0.05069387 0.02568824 0.05158759 0.05101883 0.03384547 0.03296543
 0.03958851 0.0677129  0.04548254 0.04864706 0.04599019 0.08457217
 0.05898456 0.07596384]


In [96]:
predictions = gmmresultfsh.predict_proba(filtereddata)

In [100]:
type(predictions)

numpy.ndarray

In [103]:
labels = np.array(labels)

In [104]:
filteredlabels = labels[indexestemp]

In [107]:
labelstobesent = []

for eacl in filteredlabels:
    labelstobesent.append(dictla[eacl])

In [109]:
dictla

{"'alt.atheism'": 8,
 "'comp.graphics'": 2,
 "'comp.os.ms-windows.misc'": 6,
 "'comp.sys.ibm.pc.hardware'": 9,
 "'comp.sys.mac.hardware'": 17,
 "'comp.windows.x'": 14,
 "'misc.forsale'": 18,
 "'rec.autos'": 16,
 "'rec.motorcycles'": 15,
 "'rec.sport.baseball'": 3,
 "'rec.sport.hockey'": 0,
 "'sci.crypt'": 13,
 "'sci.electronics'": 4,
 "'sci.med'": 5,
 "'sci.space'": 11,
 "'soc.religion.christian'": 19,
 "'talk.politics.guns'": 7,
 "'talk.politics.mideast'": 12,
 "'talk.politics.misc'": 1,
 "'talk.religion.misc'": 10}

In [110]:
compute_homogeneity(predictions,labelstobesent)

2.555765189164771

In [111]:
compute_completeness(predictions,labelstobesent,20,20)

2.5300046270821026

In [113]:
no_topics = 20
lda20 = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(termfq)
result20 = lda20.transform(termfq)

In [114]:
filtereddata20 = result20[indexestemp]
gmmresultfsh20 = GaussianMixture(n_components=20, covariance_type='diag').fit(filtereddata20)

In [116]:
predictions20 = gmmresultfsh20.predict_proba(filtereddata20)

In [117]:
compute_homogeneity(predictions20,labelstobesent)

2.5386416409682417

In [118]:
compute_completeness(predictions20,labelstobesent,20,20)

2.498867324812917

In [119]:
filtereddata20 = result20[indexestemp]
gmmresultfsh10 = GaussianMixture(n_components=10, covariance_type='diag').fit(filtereddata20)
predictions10 = gmmresultfsh10.predict_proba(filtereddata20)

In [120]:
compute_homogeneity(predictions10,labelstobesent)

2.6088813614416884

In [121]:
compute_completeness(predictions10,labelstobesent,10,20)

1.9151530010559497

In [125]:
no_topics = 10
lda10 = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(termfq)
result10 = lda10.transform(termfq)

In [127]:
filtereddata10 = result10[indexestemp]
gmmresultfsh1020 = GaussianMixture(n_components=10, covariance_type='diag').fit(filtereddata10)
predictions1020 = gmmresultfsh1020.predict_proba(filtereddata10)

In [128]:
compute_homogeneity(predictions1020,labelstobesent)

2.5627124977540694

In [129]:
compute_completeness(predictions1020,labelstobesent,10,20)

1.912888232611154

In [130]:
predictions10.shape
    

(4491, 10)

In [143]:
a = np.zeros(10,np.float32)

for eachdoc in predictions10:
    a = np.add(a,eachdoc) 
    

In [135]:
max1 = np.argmax(a)

In [136]:
max1

4

In [139]:
points = []
index = 0
for b in gmmresultfsh10.predict(filtereddata20):
    if(b == 4):
        points.append(index)
    index += 1

In [141]:
temp = {}
for eachtop in points:
    label = labelstobesent[eachtop]
    
    if label in temp:
        temp[label] += 1
    else:
        temp[label] = 1

In [142]:
temp

{0: 1,
 1: 1,
 2: 101,
 3: 1,
 4: 20,
 5: 4,
 6: 103,
 7: 1,
 8: 1,
 9: 107,
 10: 1,
 11: 9,
 13: 13,
 14: 140,
 15: 14,
 16: 8,
 17: 86,
 18: 43,
 19: 2}

In [147]:
list1 = [14,6,9,17,2]

In [148]:
for key in dictla:
    if dictla[key] in list1:
        print(key)

'comp.graphics'
'comp.os.ms-windows.misc'
'comp.sys.ibm.pc.hardware'
'comp.windows.x'
'comp.sys.mac.hardware'


In [152]:
import elasticsearch
es = elasticsearch.Elasticsearch()
def show_top20(classifier, vectorizer,no_top_words,type1,data):
    
    index_no = type1 + data
    doc20Ngtype = type1 + data + "type"
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(classifier.components_):
        print(i)
        
        top_words=category.argsort()[:-no_top_words - 1:-1]
        words = []
        for j in top_words:
            print(feature_names[j],category[j])
            words.append(feature_names[j])
        
        print("**************************************************************************")

In [153]:
show_top20(lda20, count_vectorizer, 5,"nmf","20ng")

0
armenian 2555.9249539253483
say 1844.6438811953346
come 1227.650433432521
israel 1181.7965708235108
win 894.9647925872529
**************************************************************************
1
child 1619.5280723395213
car 1326.641322977526
woman 1127.4291285793213
turkish 978.4817844771253
father 736.3595082790227
**************************************************************************
2
sell 953.1153968539671
price 880.8250733327653
arm 785.1483685842175
weapon 782.3359225166859
phone 768.1796294901937
**************************************************************************
3
edu 2068.5612735605805
file 2035.6539574608648
com 1191.143139689959
list 1036.0695463983207
send 1030.0753496649536
**************************************************************************
4
god 4171.172771340497
christian 1819.110738842286
believe 1614.9369557457267
jesus 1531.8484316062286
say 1504.7286288908435
**************************************************************************
5
use 1370